# 질문자
```
잘된다. 노무현으로만 했는데 모든 대통령을 각각 훈련시켜서 인자로 노무현, 국민 여러분, 우리나라의 미래는 또는 박정희, 국민 여러분, 우리나라의 미래는 이런 식으로 가능할까?
```

# Gemini
```
네, 물론 가능합니다! 그리고 그것이 바로 파인튜닝의 진정한 묘미 중 하나입니다. 한 명의 스타일에만 특화된 모델을 넘어, 하나의 모델이 여러 인물의 스타일을 학습하고, 필요에 따라 특정 인물의 스타일을 흉내 내도록 만들 수 있습니다.

사용자께서 원하시는 기능, 즉 (대통령, 시작 문장)을 인자로 받아 해당 대통령의 스타일로 뒷문장을 생성하는 것은 매우 효과적인 파인튜닝 기법으로 구현할 수 있습니다.

가장 전문적이고 널리 쓰이는 방법은 '스페셜 토큰(Special Token)'을 활용하는 것입니다.

'스페셜 토큰'을 이용한 멀티 스타일 모델 구현 전략
핵심 아이디어는 간단합니다. 모델에게 어떤 대통령의 스타일로 글을 써야 할지 알려주는 '꼬리표'를 붙여서 학습시키는 것입니다.

스페셜 토큰 정의: 각 대통령의 이름을 나타내는 특별한 토큰을 만듭니다. (예: <|노무현|>, <|박정희|>)
토크나이저 확장: 이 스페셜 토큰들을 모델이 이해할 수 있는 새로운 단어로 토크나이저에 정식 등록합니다.
데이터 전처리: 모든 학습 데이터의 맨 앞에 해당 연설을 한 대통령의 스페셜 토큰을 붙여줍니다.
학습 데이터 예시: <|노무현|> 국민 여러분, 우리나라의 미래는 우리 스스로가 만들어가야 합니다.
모델 학습: 이렇게 가공된 데이터로 모델을 파인튜닝합니다. 모델은 스페셜 토큰과 그 뒤에 이어지는 텍스트 스타일 간의 연관성을 학습하게 됩니다.
조건부 생성: 텍스트를 생성할 때, 우리가 원하는 대통령의 스페셜 토큰으로 문장을 시작하면, 모델이 해당 스타일을 따라 글을 생성하게 됩니다.
생성 요청 예시: <|박정희|> 국민 여러분, 우리나라의 미래는
```

In [1]:
import torch
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from datasets import Dataset

# --- 1. 데이터 로딩 및 스페셜 토큰 정의 ---

In [3]:
from president_speech.db.parquet_interpreter import read_parquet
df = read_parquet()
# 여기서는 모든 대통령 데이터를 사용하므로 '노무현'으로 필터링하지 않습니다.
print(f"전체 연설문 개수: {len(df)}")

전체 연설문 개수: 8771


In [4]:
# 1-1. 데이터에 있는 모든 대통령의 목록을 가져옵니다.
presidents = df['president'].unique().tolist()
print(f"학습할 대통령 목록: {presidents}")

학습할 대통령 목록: ['박정희', '최규하', '전두환', '노태우', '김영삼', '김대중', '노무현', '이명박', '이승만', '윤보선', '박근혜', '문재인']


In [5]:
# 1-2. 각 대통령을 위한 스페셜 토큰을 정의합니다.
special_tokens = [f"<|{p}|>" for p in presidents]
print(f"생성된 스페셜 토큰: {special_tokens}")

생성된 스페셜 토큰: ['<|박정희|>', '<|최규하|>', '<|전두환|>', '<|노태우|>', '<|김영삼|>', '<|김대중|>', '<|노무현|>', '<|이명박|>', '<|이승만|>', '<|윤보선|>', '<|박근혜|>', '<|문재인|>']


# --- 2. 토크나이저에 스페셜 토큰 추가 ---

In [6]:

MODEL_NAME = 'skt/kogpt2-base-v2'

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    bos_token='<s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>'
)

In [7]:
# 2-1. 정의한 스페셜 토큰들을 토크나이저의 어휘(vocabulary)에 추가합니다.
tokenizer.add_special_tokens({'additional_special_tokens': special_tokens})

12

# --- 3. 모델의 토큰 임베딩 리사이즈 ---

In [8]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

In [9]:
# 3-1. 중요: 토크나이저에 새로운 단어(토큰)가 추가되었으므로,
# 모델도 이 새로운 단어를 이해할 수 있도록 '뇌의 용량(임베딩 레이어)'을 늘려줘야 합니다.
model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(51212, 768)

# --- 4. 데이터셋 구성 방식 변경 ---

In [12]:
# 4-1. 각 연설문 텍스트 앞에 해당 대통령의 스페셜 토큰을 붙여줍니다.
formatted_texts = []
for index, row in df.iterrows():
    president_token = f"<|{row['president']}|>"
    # 예: "<|노무현|> 국민 여러분, ..."
    formatted_text = f"{president_token} {row['speech_text']}"
    formatted_texts.append(formatted_text)


In [13]:
# 4-2. 스페셜 토큰이 추가된 모든 텍스트를 하나로 결합합니다.
combined_text = "\n\n".join(formatted_texts)

In [14]:
# 4-3. Dataset 객체 생성 및 토크나이징, 청킹 (이하 과정은 이전과 동일)
raw_dataset = Dataset.from_dict({'text': [combined_text]})

In [17]:
# tokenize_and_chunk 함수는 이전 코드와 동일하게 사용합니다.
# ... (def tokenize_and_chunk ... 부분)
# 3-2. 데이터셋 토크나이징 및 청킹(Chunking)
# 텍스트를 토큰 ID로 변환하고, 긴 텍스트를 일정한 길이(block_size)의 덩어리로 자릅니다.
def tokenize_and_chunk(examples):
    # 전체 텍스트를 토크나이징합니다.
    tokenized_output = tokenizer(examples['text'], truncation=False) # 긴 텍스트이므로 자르지 않음

    block_size = 128  # 한 번에 처리할 토큰의 수 (GPU 메모리에 따라 조절)
    
    # 결과를 저장할 딕셔너리
    result = {
        "input_ids": [],
        "attention_mask": [],
        "labels": [],
    }

    # 모든 토큰을 하나로 합칩니다.
    concatenated_ids = sum(tokenized_output['input_ids'], [])
    total_length = len(concatenated_ids)

    # block_size 단위로 텍스트를 나눕니다.
    for i in range(0, total_length, block_size):
        chunk = concatenated_ids[i:i + block_size]
        
        # 마지막 청크가 block_size보다 작으면 패딩을 추가합니다.
        if len(chunk) < block_size:
            padding_length = block_size - len(chunk)
            chunk.extend([tokenizer.pad_token_id] * padding_length)
            
        result["input_ids"].append(chunk)
        # 패딩 토큰은 attention 계산에서 제외하기 위해 0으로 설정
        attention_mask = [1] * (len(chunk) - chunk.count(tokenizer.pad_token_id)) + [0] * chunk.count(tokenizer.pad_token_id)
        result["attention_mask"].append(attention_mask)
        # Causal LM 파인튜닝에서는 input_ids와 labels를 동일하게 설정합니다.
        # Trainer가 내부적으로 labels를 한 칸씩 밀어서(shift) 처리해줍니다.
        result["labels"].append(chunk)

    return result

In [16]:
lm_dataset = raw_dataset.map(tokenize_and_chunk, batched=True, remove_columns=raw_dataset.column_names)
print("멀티 스타일 파인튜닝을 위한 데이터셋 준비 완료!")

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

멀티 스타일 파인튜닝을 위한 데이터셋 준비 완료!


# --- 5. 학습 및 저장 ---

In [18]:
# TrainingArguments, DataCollator, Trainer 초기화 및 학습 과정은 이전과 완벽하게 동일합니다.
# output_dir 이름만 변경하여 이전 모델과 겹치지 않게 합니다.
training_args = TrainingArguments(
    output_dir='./all_presidents_gpt2_results', # 결과 저장 디렉터리 변경
    # ... (num_train_epochs, batch_size 등 나머지 설정은 동일)
    num_train_epochs=3,
    per_device_train_batch_size=16,
    learning_rate=5e-5,
    fp16=True,
    logging_steps=50,
    save_steps=1000,
    save_total_limit=2,
    report_to="none",
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset,
    data_collator=data_collator,
)

- 정상 결과: Trainer is configured to use device: cuda:0 와 같이 cuda가 포함된 장치 이름이 출력되어야 합니다.
- 문제 상황: 만약 Trainer is configured to use device: cpu 라고 출력된다면, Trainer가 GPU를 사용하지 않기로 결정했다는 의미입니다.
# ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼

In [19]:
# =======================================================
# ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼ 이 코드를 추가하여 확인 ▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼
print(f"Trainer is configured to use device: {trainer.args.device}")
# ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲
# =======================================================

Trainer is configured to use device: cuda:0


In [20]:
# 학습 시작
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,4.042600
100,3.980200
150,3.939800
200,3.886500
250,3.860400
300,3.837000
350,3.824400
400,3.798200
450,3.789200
500,3.795500


TrainOutput(global_step=12321, training_loss=3.322305152520916, metrics={'train_runtime': 2670.1897, 'train_samples_per_second': 73.822, 'train_steps_per_second': 4.614, 'total_flos': 1.2876340690944e+16, 'train_loss': 3.322305152520916, 'epoch': 3.0})

In [21]:
# 최종 모델 저장
final_model_path = './all_presidents_gpt2_final'
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)
print(f"멀티 스타일 모델이 '{final_model_path}'에 저장되었습니다.")

멀티 스타일 모델이 './all_presidents_gpt2_final'에 저장되었습니다.


In [22]:
# --- 6. 조건부 텍스트 생성 테스트 ---
# 저장된 멀티 스타일 모델과 토크나이저를 다시 불러옵니다.
model_ft = AutoModelForCausalLM.from_pretrained(final_model_path)
tokenizer_ft = AutoTokenizer.from_pretrained(final_model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft.to(device)

# 생성 함수를 약간 수정하여 대통령 이름을 인자로 받도록 합니다.
def generate_text_by_president(president_name, prompt, max_len=128):
    # 입력 프롬프트 앞에 해당 대통령의 스페셜 토큰을 붙여줍니다.
    president_token = f"<|{president_name}|>"
    formatted_prompt = f"{president_token} {prompt}"
    
    print(f"입력: {formatted_prompt}")

    input_ids = tokenizer_ft.encode(formatted_prompt, return_tensors='pt').to(device)
    
    gen_ids = model_ft.generate(input_ids,
                               max_length=max_len,
                               repetition_penalty=2.0,
                               pad_token_id=tokenizer_ft.pad_token_id,
                               eos_token_id=tokenizer_ft.eos_token_id,
                               bos_token_id=tokenizer_ft.bos_token_id,
                               use_cache=True,
                               do_sample=True, # 더 자연스러운 문장을 위해 샘플링 사용
                               top_k=50,
                               top_p=0.95)
    
    generated_text = tokenizer_ft.decode(gen_ids[0], skip_special_tokens=False) # 스페셜 토큰도 보이게 출력
    # 스페셜 토큰을 제외하고 싶다면 skip_special_tokens=True 로 변경
    
    return generated_text

In [24]:
# 테스트
print("\n--- 멀티 스타일 모델 생성 테스트 ---")
prompt = "북한은"

# '노무현' 스타일로 생성
print(f"생성 결과 (노무현): {generate_text_by_president('노무현', prompt)}")
print("-" * 30)

# '박정희' 스타일로 생성 (데이터에 '박정희'가 있다고 가정)
print(f"생성 결과 (박정희): {generate_text_by_president('박정희', prompt)}")
print("-" * 30)

# '김대중' 스타일로 생성 (데이터에 '김대중'이 있다고 가정)
print(f"생성 결과 (김대중): {generate_text_by_president('김대중', prompt)}")
print("--- 테스트 완료 ---")


--- 멀티 스타일 모델 생성 테스트 ---
입력: <|노무현|> 북한은
생성 결과 (노무현): <|노무현|> 북한은 그동안 미국과의 관계 정상화를 위한 각종 교류와 지원을 아끼지 않고 있으며 특히 우리의 대북 경수로 지원사업과 한반도 평화구축 사업에 대한 적극적 지지를 표명하고 있습니다. 나는 미, 북 양측이 이러한 모든 진전을 바탕으로 보다 포괄적인 차원의 대화 노력을 기울여 올 것이라고 확신하는 바입니다. 기자 올림픽을 통해 드러난 한국의 역동성과 함께 미국의 아시아 중시 정책과 한, 중 관계는 어떻게 달라지셨는지요. 대통령 저는 지난 5년여 동안 아시아, 태평양지역 국가들과의 협력관계를 크게 진전시켜 온 것을 높이 평가하며, 앞으로도 계속 노력할 것임을 약속드립니다. 미국과 중국, 일본은 아시아의 미래를 책임진 친구들이라는 사실을 잘 알고 있는데 이번 베이징올림픽에서
------------------------------
입력: <|박정희|> 북한은
생성 결과 (박정희): <|박정희|> 북한은 지금도 우리 정부를 전복시키려는 전쟁 도발을 조금도 하지 않고 있습니다. 그들은 적화 통일을 기도하며 온갖 책동과 수단과 방법을 가리지 않는 것입니다. 지난 1년 사이에도 휴전선을 사이에 둔 남침용 땅굴을 파고 있는 것은 바로 그들의 반민족적 흉계임을 금할 수가 없습니다. 이제 북한 침략주의자들은 우리의 남북 관계 개선 노력을 외면하는 한 언제 어디서 어떠한 형태의 무력도발이나 폭력 만행을 할 것으로 예측됩니다. 이러한 일이 일어난다면, 우리는 그 누구도 넘볼 수는 없는 것이며, 만일 북한의 이같은 무모한 군사적 모험이 성공한다면 그들이 가지고 있을 여러 가지 후방 교란 공작 등 모든 수단을 동원해 단호히 응징해야 하겠
------------------------------
입력: <|김대중|> 북한은
생성 결과 (김대중): <|김대중|> 북한은 금년에 들어와서 유엔, 미국, 유럽연합 등 3대 국제기구에 가입하는 나라들로 들어가고 있습니다. 이번에 북한과의 대화